<a href="https://colab.research.google.com/github/emailic/YOLOv5-Microplasticos/blob/main/detect%26recompose_img1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import os
import pandas as pd

from collections import namedtuple
from PIL import Image, ImageFont, ImageDraw, ImageEnhance
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [21]:
Rectangle = namedtuple('Rectangle', 'xmin ymin xmax ymax label conf')
color_dict = ["green", "blue", "red", "yellow", "orange"]
OUT_COLS = [
    "image_path",
    "xmin",
    "ymin",
    "xmax",
    "ymax",
    "label",
    "confidence",
    "x_size",
    "y_size",
]

In [36]:
def paint_rectangles(rectangles, grid_image, out_img, classes):
    draw = ImageDraw.Draw(grid_image)

    for rect in rectangles:
        draw.rectangle([(rect.xmin, rect.ymin), (rect.xmax, rect.ymax)], outline=color_dict[rect.label])
        draw.text((rect.xmin, rect.ymin - 10), "" + str(classes[rect.label]) + ": " + str(rect.conf), fill=color_dict[rect.label])

    grid_image.save(out_img)

In [23]:
def intersection_percentage(rect1, rect2):
    ia = intersection_area(rect1, rect2)
    if (ia != None and ia > 0.0):
        rect1_area = (rect1.xmax - rect1.xmin) * (rect1.ymax - rect1.ymin)
        rect2_area = (rect2.xmax - rect2.xmin) * (rect2.ymax - rect2.ymin)
        percentage = ia / (rect1_area + rect2_area - ia)
        
        return percentage if percentage != None else 0
    else:
        return 0

# returns None if rectangles don't intersect
def intersection_area(rect1, rect2):  
    dx = min(rect1.xmax, rect2.xmax) - max(rect1.xmin, rect2.xmin)
    dy = min(rect1.ymax, rect2.ymax) - max(rect1.ymin, rect2.ymin)
    if (dx>=0) and (dy>=0):
        return dx*dy
    else:
        return 0 

In [24]:
def generate_rectangles(csv_row): #positions rectangles on a full sized image
    [h_offset, w_offset]= csv_row["grid_coords"].split(", ")
    xmin = int(w_offset) + int(csv_row["xmin"])
    xmax = int(w_offset) + int(csv_row["xmax"])

    ymin = int(h_offset) + int(csv_row["ymin"])
    ymax = int(h_offset) + int(csv_row["ymax"])

    return Rectangle(xmin, ymin, xmax, ymax, int(csv_row["class"]), float(csv_row["confidence"]))

In [41]:
def rebuild_image(csv_path, out_csv_path, out_img, full_img_path, classes, treshold=0.5): #creates an output detection .csv file and prints the image with label boxes
    detect_csv = pd.read_csv(csv_path)
    out_df = pd.DataFrame(
        columns=OUT_COLS
    )
    total_rectangles = [generate_rectangles(row) for _, row in detect_csv.iterrows()]
    grid_image = Image.open(full_img_path, 'r')
    
    rectangles = []
    for i, rect1 in enumerate(total_rectangles):
        j = 0
        if rect1.conf < treshold: 
            continue
        while j < len(total_rectangles):
            if j == i: 
                j += 1
                continue
            rect2 = total_rectangles[j]
            intersection = intersection_percentage(rect1, rect2)
            if intersection > 0.25 and rect1.conf <= rect2.conf:
                break
            j += 1

        if j == len(total_rectangles):
            rectangles.append(rect1)
            out_df = out_df.append(
                pd.DataFrame(
                    [
                        [
                            out_img,
                            rect1.xmin,
                            rect1.ymin,
                            rect1.xmax,
                            rect1.ymax,
                            rect1.label,
                            rect1.conf,
                            rect1.xmax - rect1.xmin, 
                            rect1.ymax - rect1.ymin
                        ]
                    ],
                    columns=OUT_COLS
                )
            )

    out_df.to_csv(out_csv_path, index=False)
    print("Rectangles: ", str(len(rectangles))) #number of counted items on the image
    print("Total Rectangles: ", str(len(total_rectangles)))
    paint_rectangles(rectangles, grid_image, out_img, classes)

In [42]:

csv_path='/content/drive/My Drive/microplasticos/microplasticos_1152/csvs/20220516_113718.jpg.csv'
out_img= '/content/drive/My Drive/microplasticos/microplasticos_1152/results/detection1/20220516_113718.jpg'
out_csv_path= out_img+'_OUT.csv'
img_path='/content/drive/My Drive/microplasticos/full_imgs_jsons'
img_name='20220516_113718.jpg'
img_file = os.path.join(img_path, img_name)
classes=['fragment', 'line', 'organic', 'pellet', 'tar']

rebuild_image(csv_path, out_csv_path, out_img, img_file, classes) #(csv_path, out_csv_path, out_img, classes, full_img_path, treshold=0.5)

Rectangles:  67
Total Rectangles:  555
